In [88]:
#Pacotes que estamos importando
import numpy as np               
import pandas as pd             
from scipy.stats import rankdata # Para o ranking das candidatas

In [163]:
# Dados

attributes = np.array(["Capacidade", "Autonomia", "Eficiência", "Preço"])
candidates = np.array(["DJ", "Pelicano", "Dractor"])
raw_data = np.array([
    [40, 102,  21.3,  229999],
    [30, 68,  9.3,  170000],
    [10, 240,  6,   159999],
    
])

#print(raw_data)

weights = np.array([0.125, 0.125, 0.25, 0.5])

# Indices das alternativas de lucro (contagem iniciando do zero).
# Indices não listados são considerados de custo.
benefit_attributes = set([0,1,2])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

#Step 1
#Normalização

m = len(raw_data)
n = len(attributes)

#Criando um vetor de tamanho n fazio e logo em seguida estou preenchendo cada vetor com o tamanho máximo da coluna 
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.max(column)

#Dividindo cada termo pelo divisor
raw_data /= divisors

#Matriz de Decisão
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)


,Capacidade,Autonomia,Eficiência,Preço
DJ,1.00,0.425000,1.00000,1.000000
Pelicano,0.75,0.283333,0.43662,0.739134
Dractor,0.25,1.000000,0.28169,0.695651


In [164]:
#Step 2

# Criando uma matriz com todas as colunas zerados com o comprimento dos critérios
all_columns = np.zeros((raw_data.shape[0], len(attributes)))

# Preenchendo as colunas de lucro com os valores correspondentes
all_columns[:, list(benefit_attributes)] = raw_data[:, list(benefit_attributes)]

#print(all_columns)

benefit_profit = all_columns*weights

#print(benefit_profit)
cost_profit = (raw_data - all_columns)*weights

#Pra não atrapalhar, vou tirar as colunas que não são zeros 

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(benefit_profit != 0, axis=0) 
benefit_profit = benefit_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

#print(benefit_profit)

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(cost_profit != 0, axis=0) 
cost_profit = cost_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

#print(cost_profit)

column_sums = np.sum(benefit_profit, axis=1)
column_sums2 =np.sum(cost_profit, axis=1)
column_products = np.prod(benefit_profit, axis=1)
column_products2 = np.prod(cost_profit, axis=1)
u_i_wsd = column_sums - column_sums2
u_i_wpd = column_products- column_products2

#Aqui vou fazer um if para caso só tenhamos critérios de lucro ou só critério de custo:

if np.any(cost_profit) and np.any(benefit_profit):# Ambas as matrizes são diferentes de zero
    u_i_wsr = column_sums/column_sums2
    u_i_wpr = column_products/column_products2
    
# A matriz cost_profit é diferente de zero e a matriz benefit_profit é igual a zero
#Ou seja só tem critério de custo
elif np.any(cost_profit) and not np.any(benefit_profit):
    u_i_wsr = 1/column_sums2
    u_i_wpr = 1/column_products2

#Aqui estamos com cost_profit igual a 0
else :
    
    u_i_wsr = column_sums
    u_i_wpr = column_products

#print(cost_profit)
#print(benefit_profit)
#print(row_sums)
#print(row_sums2)
#print(benefit_profit)
#print(cost_profit)
#print(column_sums)
#print(column_sums2)

#print(column_products)
#print(column_products2)
#print(column_products)
#print(column_products2)

#print(u_i_wsd)
#print(u_i_wpd)
#print(u_i_wsr)
#print(u_i_wpr)

df = pd.DataFrame({
    'u_i_wsd': u_i_wsd,
    'u_i_wpd': u_i_wpd,
    'u_i_wsr': u_i_wsr,
    'u_i_wpr': u_i_wpr
},index=candidates)

# Exibir o DataFrame
df

,u_i_wsd,u_i_wpd,u_i_wsr,u_i_wpr
DJ,-0.071875,-0.498340,0.856250,0.003320
Pelicano,-0.131245,-0.369204,0.644867,0.000981
Dractor,-0.121153,-0.347550,0.651685,0.000791


In [166]:
#Step 3

u_i_wsd2 = u_i_wsd/(1+np.max(u_i_wsd))
u_i_wpd2 = u_i_wpd/(1+np.max(u_i_wpd))
u_i_wsr2 = u_i_wsr/(1+np.max(u_i_wsr))
u_i_wpr2 = u_i_wpr/(1+np.max(u_i_wpr))

df2 = pd.DataFrame({
    'u_i_wsd2': u_i_wsd2,
    'u_i_wpd2': u_i_wpd2,
    'u_i_wsr2': u_i_wsr2,
    'u_i_wpr2': u_i_wpr2
}, index = candidates)


df2

,u_i_wsd2,u_i_wpd2,u_i_wsr2,u_i_wpr2
DJ,-0.077441,-0.763798,0.461279,0.003309
Pelicano,-0.141409,-0.565874,0.347403,0.000977
Dractor,-0.130535,-0.532685,0.351076,0.000788


In [180]:
column_sums = df2.sum(axis=1)/4

#Fazendo o ranquamento
ranked_column_sums = column_sums.rank(ascending=False).sort_values(ascending=True).reset_index()
ranked_column_sums = ranked_column_sums.rename(columns={'index': 'Alternativa', 0: 'Rank'})

# Exibir o resultado do ranqueamento formatado
print(ranked_column_sums)

  Alternativa  Rank
0     Dractor   1.0
1    Pelicano   2.0
2          DJ   3.0
